# 1. About simulation of models and trajectories

The goal of this notebook is to present the basics of the simulation in our package.

Let's get familiar with the package. First, we shoud load it.

In [ ]:
using ABCRN

## Models

In this package, we are focused on Continuous-Time Markov Chains models that can be described by Chemical Reaction Networks.

Let's simulate our first model. We consider the famous SIR epidemiology model described by two reactions:

$$
Infection: S + I \xrightarrow{k_i} 2I \\
Recovery: I \xrightarrow{k_r} R
$$

In the ABCRN package, models are objects and their types all derived from the abstract type `Model`. Let's load the SIR model.

In [ ]:
load_model("SIR")

This function searchs a model file called SIR.jl in the models/ directory (at the root of the package). This file creates the necessary resources to create a `ContinuousTimeModel` and store it in a variable called SIR.

In [ ]:
println(SIR)

This variable contains a parameter vector and an initial point. It is ready to use.

In [ ]:
@show SIR.p
@show SIR.x0

You can change the parameters with the function `set_param!`

In [ ]:
set_param!(SIR, :ki, 0.015)
@show SIR.p
set_param!(SIR, [0.02, 0.07])
@show SIR.p

## Trajectories

The simulation of the model is done by the function `simulate`.

In [ ]:
σ = simulate(SIR)

`simulate` returns a path which type is derived from `AbstractTrajectory`. It can be either an object of type `Trajectory` for models `::ContinuousTimeModel` or `SynchronizedTrajectory` for models that includes an automaton (but this is the subject of another notebook). It is easy to access the values of a trajectory:

In [ ]:
@show σ[3] # the third state of the trajectory
@show length_states(σ) # number of states
@show σ[:I, 4] # Fourth value of the variable I
@show σ.I[4] # Fourth value of the variable I
@show get_state_from_time(σ, 2.3)

The SIR object includes an observation model symbolized by the vector `SIR.g`. Even if the variables of the model are `[:S, :I, :R]`, only I will be observed.

In [ ]:
@show SIR.map_var_idx
@show SIR.g
@show size(σ.values), length(σ[:I]) # Only one column which corresponds to the I variable

The SIR model is by default unbounded, i.e. each trajectory is simulated until it reaches an absorbing state.

In [ ]:
@show isbounded(SIR)
@show isbounded(σ)

For example, computations of Lp integral distances needs bounded trajectories.

In [ ]:
dist_lp(simulate(SIR),simulate(SIR); p=2)

But this feature can be changed.

In [ ]:
set_time_bound!(SIR, 120.0)

In [ ]:
@show dist_lp(simulate(SIR),simulate(SIR))
@show isbounded(SIR)
@show isbounded(simulate(SIR))

## Buffer size

A useful feature is that we can change the preallocation of the memory size during the simulation in order to gain computational performance.

In [ ]:
load_model("ER")
@show ER.buffer_size
@timev σ = simulate(ER)
@show length_states(σ)

By default, the buffer size is 10. It means that a matrix of size 10 is allocated anf filled. When it is full, another matrix of size 10 is allocated. But if you know that your simulations will have a certain number of states, you can change the buffer size. It can make a big difference in terms of performance.

In [ ]:
ER.buffer_size = 100
@timev σ2 = simulate(ER)
@show length_states(σ2)

## Plots